<h1><center>Meera Retailer Campaign - Random Pick</center></h1>

## Objective
* Choose random retailers satisfying certain criteria providing weights based on the purchasing ability of retailers
* Choose the number of random retailers based on the weightage of number of retailers under each AM

In [ ]:
from ckpackages import azsql 
import numpy as np
import pandas as pd
import random
import os
from glob3 import glob

In [3]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import Button, Output, VBox,interactive,interact

In [6]:
## Read & join csv in same folder
a = os.getcwd()
path = a + '\\Outputs_csv'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
df_concat = pd.concat(li, axis=0, ignore_index=True)

# Convert InvDate to Date format
df_concat['InvDate'] = pd.to_datetime(df_concat['InvDate'])

## Filter data by instances occuring in last week
days=6 #Last 6 days from current date
cutoff_date = df_concat['InvDate'].max() - pd.Timedelta(days=days)
print(cutoff_date)

df1_concat = df_concat[df_concat['InvDate'] > cutoff_date]

# List of last 6 days unique retailers
rtr_last6days = df1_concat[['RtrCode','RtrName']].drop_duplicates()
rtr_last6days = rtr_last6days['RtrCode'].tolist()

# Filter retailer occuring in last 6 days (to be included below)
#data_filtered = data[~data.RtrCode.isin(rtr_last6days)]

AttributeError: 'function' object has no attribute 'glob'

In [98]:
## Import data
Query = 'select * from v_PC_MeeraRetailerCampaign' 
data = azsql.callstatement(Query)

# Data only for Latest date
data = data[data['InvDate'] == data['InvDate'].max()]

## Remove null - Retailers without AM
# Consider Data with AMName ( remove nulls)
data = data[data['AMName'].notnull()]

# Filter retailer occuring in last 6 days
#data = data[~data.RtrCode.isin(rtr_last6days)]

InvDate =  data['InvDate'].iloc[0]
InvDate = str(InvDate)[0:10]

## Retailer master
rtr_master = data[['RtrCode','RtrName','DistName','RtrGroupCde','InvDate','PolypouchCount']].drop_duplicates()
rtr_master['PolypouchCount'] = rtr_master['PolypouchCount'].astype(int)

AM_RTR_in = data.groupby('AMName')['RtrCode'].count().reset_index()

# Create text widget for output
output_slider_variable = widgets.Text()

# Define function to bind value of the input to the output variable 
def f(Lots):
    output_slider_variable.value = str(Lots)

print('Choose number of Lots')
# Create input slider with default value = 10  
interact(f, Lots=(10, 75, 1))

def picks(output_slider_variable,data):
    # Create and output new int variable with value of slider
    new_variable = int(output_slider_variable.value)

    ## Choose number of retailers to choose
    ## Weightage for AM
    # Percentage of Retailers across AM-Wise
    TotalRtrCount = data['RtrCode'].count()
    AM_RTR_WGT = data.groupby('AMName')['RtrCode'].count().reset_index()
    AM_RTR_WGT['WGT%'] = np.ceil(AM_RTR_WGT['RtrCode']/TotalRtrCount * new_variable)

    ## Split dataframe on AM's
    # Select required rows
    data1 = data[['ZM','AMName','RtrCode','RtrName','TotalQty','PolypouchCount']]
    data2 = [rows for _, rows in data1.groupby('AMName')]
    return data2,AM_RTR_WGT
    
## Function to generate random Rtr's with AM weightage
def randomRtrAmWise(i):
    data3,AM_RTR_WGT = picks(output_slider_variable,data)
    # Retailer name
    a = data3[i]['RtrCode'].tolist()

    # Polypouch count as weight
    ppc = data3[i]['PolypouchCount'].tolist() 

    c=int(AM_RTR_WGT['WGT%'][i])
    #random.shuffle(a)
    l = random.choices(a, weights=ppc, k=c)
    l.sort()
    df_R = pd.DataFrame(l,columns = ['RtrCode'])
    df_R['AMName'] = AM_RTR_WGT['AMName'][i]
    return df_R
    
def randompick(AM_RTR_in):
    df_RtrRP = pd.DataFrame()
    for j in range(len( AM_RTR_in.index.tolist())):
        df = randomRtrAmWise(j)
        df_RtrRP = df_RtrRP.append(df)
    return df_RtrRP

def randompickAM(rtr_master,AM_RTR_in,InvDate):
    result = randompick(AM_RTR_in)
    result = result.merge(rtr_master,on='RtrCode')
    result = result.groupby('RtrCode').head(2).reset_index(drop=True) # Keep 1st two instances of duplicates
    index = result.index
    number_of_rows = len(index)
    print('Count of picked retailers: ',number_of_rows)
    display(result)
    return result,InvDate
    
button = Button(description='Pick')
out = Output()

def on_click(_):
    with out:
        clear_output(True)
        dr,InvDate1 = randompickAM(rtr_master,AM_RTR_in,InvDate)
        # Save results
        root = 'Intermediate_csv' 
        sheet = 'results_' + InvDate1 + '.csv' 
        file_name = os.path.join(root,sheet)
        print(file_name)
        dr.to_csv(file_name,index=False)
        
button.on_click(on_click)
VBox([button,out])

Choose number of Lots


interactive(children=(IntSlider(value=42, description='Lots', max=75, min=10), Output()), _dom_classes=('widge…